In [5]:
import gevent
import gevent.monkey
# gevent.monkey.patch_all()

In [6]:
from core.utils import (
    PsqlQuery,
    PsqlAbstract,
    clean_comment, clean_query,
    query2halfwidth
)
import json
import re
import time
from core.ingest import PsqlIngester
from configparser import RawConfigParser
config_parser = RawConfigParser()
config_parser.read('../config.ini')

from datetime import datetime

mock_data = {
  "title": "失憶了該怎麼辦？",
  "tag": "問卦",
  "spider": "gossiping",
  "url": "https://www.ptt.cc/bbs/Gossiping/M.1502013732.A.344.html",
  "author": "shanpinlo ((((((((((()))))))))))",
  "push": "james732: 得了失億可能對你我都好QQ\ncake10414: .....\nZNDL: 哪部？\nA0091127: 先尻一槍\nyuugen2: 你被學長肛的經驗之談嗎\nbenkk41: 甲味",
  "publish_date": "2017-08-06T18:02:09+00:00"
}

    

In [7]:
PsqlAbstract.set_database_info(
    config_parser.get('global', 'dbuser'),
    config_parser.get('global', 'dbname'),
    config_parser.get('global', 'dbpassword')
)


In [22]:
class OkbotJLParser(object):
    def __init__(self, jlpath):
        self.jlpath = jlpath
    
    def batch_parse(self, batch_size=10):
        with open(self.jlpath, 'r') as f:
            i = 0
            parsed = []
            for line in f:
                parsed.append(self._parse(line))
                i += 1
                if i >= batch_size:
                    i = 0
                    yield [ps for ps in parsed if bool(ps)]
                    parsed = []

            yield [ps for ps in parsed[:i] if bool(ps)]


    def _parse(self, line):
        try:
            post = json.loads(line)
            url = post['url']
            post['date'] = datetime.strptime(post['publish_date'], '%Y-%m-%dT%H:%M:%S+00:00')
            return post
        except Exception as e:
            print(e)
#             logger.warning(e)
#             logger.warning('command okbot_ingest, jsonline record faild to parse in, ignored. line: {}'.format(line.encode('utf-8').decode('unicode-escape')))
            return {}


def subtask(post, vocab_comment):
    batch_comment = post['batch_comment']
    cmt_bundle = ingester.insert_comment(batch_comment, [post['url']])
    vocab = ingester.insert_vocab_ignore_docfreq(batch_comment, tokenized_field='comment_tokenized')
    ingester.upsert_vocab2comment(batch_comment, vocab, cmt_bundle)
    vocab_comment.extend(vocab)

def task(batch_post):
    post_url = ingester.upsert_post(batch_post)
    vocab_post = ingester.insert_vocab_ignore_docfreq(batch_post)
    ingester.upsert_vocab2post(batch_post, vocab_post, post_url)
    ingester.insert_title(batch_post, post_url)
    vocab_comment = []
    jobs = [gevent.spawn(subtask, post, vocab_comment) for post in batch_post]
    gevent.joinall(jobs)
#     for post in batch_post:
#         batch_comment = post['batch_comment']
#         cmt_bundle = ingester.insert_comment(batch_comment, [post['url']])
#         vocab = ingester.insert_vocab_ignore_docfreq(batch_comment, tokenized_field='comment_tokenized')
#         ingester.upsert_vocab2comment(batch_comment, vocab, cmt_bundle)
#         vocab_comment.extend(vocab)
    vocab_comment = list(set(vocab_comment))
    ingester.update_vocab_postfreq(vocab_post)
    ingester.update_vocab_commentfreq(vocab_comment)

In [9]:
parser = OkbotJLParser('formatted.jl')
ingester = PsqlIngester('jieba')

In [24]:
start = time.time()
start_ = start
jobs = []
for idx, batch_post in enumerate(parser.batch_parse(batch_size=1)):
    if len(batch_post) > 0:
        task(batch_post)
#         jobs.append(gevent.spawn(task, batch_post))
        if idx % 50 == 3:
#             gevent.joinall(jobs)
#             jobs = []
            break
# gevent.joinall(jobs)

print('Elapsed time @total: {:.2f}sec.'.format(time.time() - start_))


Elapsed time @total: 1.20sec.


In [4]:
post = {'author': 'forget ()',
 'batch_comment': [{'audience': 'eatingshit',
   'comment': 'high',
   'comment_tokenized': [CustomTok('high', 'eng')],
   'ctype': 'text',
   'floor': 6},
  {'audience': 'janbun999',
   'comment': '幫高調',
   'comment_tokenized': [CustomTok('幫高', 'v'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 1},
  {'audience': 'ducantimmy',
   'comment': '高調高調',
   'comment_tokenized': [CustomTok('高', 'a'), CustomTok('調高', 'v'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 7},
  {'audience': 'ota978',
   'comment': '高調',
   'comment_tokenized': [CustomTok('高', 'a'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 4},
  {'audience': 'ota978',
   'comment': '高調',
   'comment_tokenized': [CustomTok('高', 'a'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 10},
  {'audience': 'tfhkrt',
   'comment': '幫高',
   'comment_tokenized': [CustomTok('幫高', 'v')],
   'ctype': 'text',
   'floor': 2},
  {'audience': 'kenfu0402',
   'comment': '高',
   'comment_tokenized': [CustomTok('高', 'a')],
   'ctype': 'text',
   'floor': 3},
  {'audience': 'Paul0041',
   'comment': '高調',
   'comment_tokenized': [CustomTok('高', 'a'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 13},
  {'audience': 'lee16818',
   'comment': '高調',
   'comment_tokenized': [CustomTok('高', 'a'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 11},
  {'audience': 'CIRCAshi',
   'comment': '高',
   'comment_tokenized': [CustomTok('高', 'a')],
   'ctype': 'text',
   'floor': 12},
  {'audience': 'blackchc',
   'comment': '幫高調',
   'comment_tokenized': [CustomTok('幫高', 'v'), CustomTok('調', 'vn')],
   'ctype': 'text',
   'floor': 9},
  {'audience': 'quester08',
   'comment': '高',
   'comment_tokenized': [CustomTok('高', 'a')],
   'ctype': 'text',
   'floor': 5}],
 'comment_cleaned': 'janbun999: 幫高調\ntfhkrt: 幫高\nkenfu0402: 高\nota978: 高調\nquester08: 高\neatingshit: high\nducantimmy: 高調高調\nblackchc: 幫高調\nota978: 高調\nlee16818: 高調\nCIRCAshi: 高\nPaul0041: 高調',
 'comment_raw': 'janbun999: 幫高調\ntfhkrt: 幫高\nkenfu0402: 高\nota978: 高調\nquester08: 高\neatingshit: high\nducantimmy: 高調\nducantimmy: 高調\nblackchc:  幫高調\nota978: 高調\nlee16818: 高調\nCIRCAshi: 高\nPaul0041: 高調',
 'ctype': 'text',
 'date': datetime(2017, 8, 6, 9, 5, 9),
 'tag': '問卦',
 'title_cleaned': '朋友弟弟失蹤,請高屏地區的朋友幫忙',
 'title_raw': '朋友弟弟失蹤，請高屏地區的朋友幫忙',
 'title_tokenized': [CustomTok('朋友', 'n'),
  CustomTok('弟弟', 'n'),
  CustomTok('失', 'v'),
  CustomTok('蹤', 'zg'),
  CustomTok(',', 'x'),
  CustomTok('請', 'v'),
  CustomTok('高', 'a'),
  CustomTok('屏', 'ng'),
  CustomTok('地區', 'n'),
  CustomTok('的', 'uj'),
  CustomTok('朋友', 'n'),
  CustomTok('幫忙', 'v')],
 'url': 'https://www.ptt.cc/bbs/Gossiping/M.1501981512.A.204.html'}
ingester = PsqlIngester('gossiping', 'jieba')

In [6]:
batch_comment = post['batch_comment']
cmt_bundle = ingester.insert_comment(batch_comment, [post['url']])
vocab_bundle = ingester.insert_vocab_ignore_docfreq(batch_comment, tokenized_field='comment_tokenized')
# ingester.upsert_vocab2comment(batch_comment, vocab, cmt_bundle)

qvocab, vschema = ingester.query_vocab(vocab_bundle)
qcomment, cmtschema = ingester.query_comment(cmt_bundle)
tokenized = [[(k.word, k.flag, 'jieba') for k in p['comment_tokenized']] for p in batch_comment]
for vocab in qvocab:
    vtuple = (vocab[vschema['word']], vocab[vschema['pos']], vocab[vschema['tokenizer']])
#     print(tokenized)
    comment_id_with_vocab = [cmt[cmtschema['id']] for idx, cmt in enumerate(qcomment) if vtuple in tokenized[idx]]
# comment_id_with_vocab = [cmt[cmtschema['id']] for idx, cmt in enumerate(qcomment) if vtuple in tokenized[idx]]

In [15]:
s=list(set(vocab_comment))

In [16]:
len(s), s[:10]

(20666,
 [('報警', 'n', 'jieba'),
  ('試過', 'v', 'jieba'),
  ('反對', 'v', 'jieba'),
  ('精', 'a', 'jieba'),
  ('法令', 'n', 'jieba'),
  ('錫', 'n', 'jieba'),
  ('有罪', 'vn', 'jieba'),
  ('巔峰', 'n', 'jieba'),
  ('投美', 'vn', 'jieba'),
  ('聊', 'v', 'jieba')])